## 002 - Map matching process
#### This script map matches the GPS points (sequences of points that represent the trajectories) to the road network represented as a vector file, and calculates the average speed and frequency per road segment. This codes uses the map matching algorithm developed by Nimol Vamoeurn (nimolva@gmail.com) and adapts it for this research purposes. The code uses a list of csv files as inputs that are produced by the code 001. 

In [ ]:
import mapmatcher as mm
import os
import csv
import itertools
from collections import Counter, defaultdict
from datetime import datetime
#spatial libraries 
import fiona
from shapely.geometry import shape, Point, mapping
from shapely.ops import linemerge
import pandas as pd 

startTime = datetime.now()

In [ ]:
# Read Road Network data from Shapefile 
street_network = 'C:/Data/road_network_melb_prj.shp'
# Set segmment ID - OBJECTID 
seg_info = mm.getSegmentInfo(street_network)
seg_geom = mm.getSegmentShapes(street_network)
graph = mm.getNetworkGraph(street_network, seg_info[1])
endpoints = seg_info[0]
length = seg_info[1]
# Create a spatial index for the road network
idx = mm.buildRTree(street_network)


# Input 
in_dir = 'C:/Data/Input files/test/'
# Read point data
csv_input = os.path.join(in_dir,r'test_am.csv')
# Output 
out_file = 'C:/Data/Output files/test_am.csv'


print "Network and data loaded "
print (datetime.now() - startTime)

In [ ]:
def FloatOrZero(value):
    try:
        return float(value)
    except:
        return 0.0

In [ ]:
with open(csv_input, 'r') as input_track:
    track = csv.reader(input_track, delimiter=',')
    header_line = next(track)
    seg_count_list = []
    speed_count = defaultdict(list)
    seg_speed_avg = {}
    counter = 0
    counter_sessions = 0
    for key, group in itertools.groupby(track, key=lambda x: x[6]):
        try:
            indiv_track = []
            speed_track = [float(0)]
            print key
            print "------------------"
            for p in group:
                # x and y coordinates
                indiv_track.append((float(p[3]), float(p[4])))
                if p[8] not in (None, ""):
                    speed_track.append(float(p[8]))
            print indiv_track
            opt = mm.mapMatch(indiv_track, seg_info, graph, idx, seg_geom,
                                  500, 400, 50)
            print opt
            # Create list of all matched segments
            for seg in opt:
                seg_count_list.append(seg)
  
            #Clean the path (remove double segments and crossings, etc.)
            opt_clean = mm.cleanPath(opt, endpoints)
        
            print (" %d total segments matched for segment ID" %(len(opt)))
            print (" %d cleaned segments matched for segment ID" %(len(opt_clean)))
            print (" %d records for session id %s" %((len(indiv_track)), key))
            print (" %d valid speed records for session id %s") %((len(speed_track)), key)
            #Append speed value for matched segment 
            for segment, value in itertools.izip(opt, speed_track):
                if segment in opt_clean:
                    speed_count[segment].append(value)
                    
            counter_sessions +=1
        except:
            counter += 1
    
    #Calculate segment frequency 
    seg_freq = {x: seg_count_list.count(x) for x in seg_count_list}
    #Calculate average speed for each segment
    for items in speed_count.items():
        seg_speed_avg[items[0]] = sum(items[-1])/len(items[-1])
                      

In [ ]:
# Merge data 
df_freq = pd.DataFrame(seg_freq.items()) 
df_freq.rename(columns={0: 'SegmentID', 1: 'Freq'}, inplace=True)
print (df_freq.head())
df_speed = pd.DataFrame(seg_speed_avg.items())
df_speed.rename(columns={0: 'SegmentID', 1: 'Avg_speed'}, inplace=True)
print (df_speed.head())


In [ ]:
df_segments = df_freq.join(df_speed.set_index('SegmentID'), on = 'SegmentID')
df_segments

In [ ]:
df_segments.to_csv(out_file,encoding = 'utf-8')

In [ ]:
print ("Done!")
print "Summary:"
print "-------------------------------------"
print ("%d matched segments" %(len(seg_freq.keys())))
print ("%d total mapped sessions" %(counter_sessions))
print ("%d broken or unmappable sessions" %(counter))
print (datetime.now() - startTime)